## Spacy Lemma + MVP

This model will incorporate a lemmatization of the questions, by spaCy, to see if this improves upon on the MVP model. Regardless, the results will be scrutinized to determine if any patterns can be established of the pairs which are signficantly mis-classified.

**Pipeline**:
1. Stack questions
2. Lemmatize questions
3. TF-IDF
4. NMF (5 topics)
5. Unstack questions
6. XGBoost classifier

In [1]:
# data manipulation
import utils
import pandas as pd
import numpy as np

# modeling
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier

In [2]:
train_df = utils.load('train')

In [3]:
X = utils.stack_questions(train_df)

In [4]:
docs = utils.cleanup_text(X) # 42 min

In [7]:
type(docs[0])

numpy.str_

In [5]:
vecs = utils.create_vectors(docs)## need to fix the error, but not needed for this model

TypeError: Argument 'string' has incorrect type (expected str, got numpy.str_)

In [11]:
pipe = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('lemma', FunctionTransformer(utils.cleanup_text, validate=False)),
        ('tfidf', TfidfVectorizer()),
        ('nmf', NMF(n_components = 5)),
        ('unstack', FunctionTransformer(utils.unstack_questions, validate=False)),
        ('xgb', XGBClassifier(n_estimators=500, random_state=42))
    ]
)

y = train_df.is_duplicate.values
pipe.fit(train_df, y)

Pipeline(memory=None,
     steps=[('stack', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function stack_questions at 0x1a1a389158>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('lemma', FunctionTransformer(accept_sparse=False, ch...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [12]:
results_df = utils.load('results')
results_df

,accuracy,precision,recall,f1,auc,log_loss
"mvp (tf-idf, nmf(5), xgboost)",0.704833,0.670638,0.39413,0.496481,0.74804,0.563257


In [13]:
# this takes a long time, related to the lemmatization process
results_df = results_df.append(utils.log_scores(pipe, train_df, y, 'mvp (+ lemma)'))
results_df

,accuracy,precision,recall,f1,auc,log_loss
"mvp (tf-idf, nmf(5), xgboost)",0.704833,0.670638,0.394130,0.496481,0.748040,0.563257
mvp (+ lemma),0.698585,0.654416,0.389139,0.488060,0.742691,0.568667


Very similar results compared to the MVP. Need to analyze the pairs which are difficult to classify and determine the next steps.

### If I tokenzie my training data, then do I need to do the same for test data when predicting?